<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from Akvo import Flow
from selenium import webdriver
from time import sleep
import os.path
import pandas as pd
import urllib.parse
import requests as r
import json

In [2]:
instanceURL = "https://anu.akvolumen.org/api"
workdir = "../charts-generator/"
jsondir = "json/"
cachedir = "cache/"
templatedir = "template/"
server = "http://localhost:4000/"

In [3]:
dboard = Flow.getData(instanceURL + '/library', Flow.getToken())
visual = dboard.get('visualisations')
df = pd.DataFrame(visual)
sl = pd.read_csv('list-of-figures.csv')
cm = sl[sl['source'] == 'Custom']
sl = sl[sl['source'] != 'Custom']

In [4]:
df['specstr'] = df['spec'].astype('str')

In [5]:
def parseString(x):
    x = x.replace("'",'"').replace('None','null').replace('False','false').replace('True','true')
    x = urllib.parse.unquote(x)
    x = urllib.parse.quote(x)
    return x

In [6]:
df['specurls'] = df['specstr'].apply(parseString)
df['endpoints'] = '/aggregation/' + df['datasetId'] + '/' + df['visualisationType'] + '?query=' + df['specurls']

In [7]:
df = df.merge(sl, left_on='id', right_on='source')

In [8]:
ds = df[['Fid','Fname','endpoints','visualisationType','id']]

In [9]:
ds = ds.rename(columns={
    'Fid':'id',
    'Fname':'name',
    'endpoints':'endpoint',
    'visualisationType':'type',
    'id':'lumenid'
})

In [10]:
# Debug
# ds['endpoint'][1]
# testendpoint = '/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2518/bar?query=%7B%22filters%22%3A%20%5B%5D%2C%20%22sort%22%3A%20null%2C%20%22subBucketColumn%22%3A%20%22c26020003%22%2C%20%22showLabels%22%3A%20false%2C%20%22metricAggregation%22%3A%20%22count%22%2C%20%22axisLabelXFromUser%22%3A%20false%2C%20%22legendTitle%22%3A%20%22%285.1.26%29%20Do%20you%20think%20that%20the%20winning%20candidate%20will%20be%20able%20to%20find%20out%20afterwards%20who%20you%20voted%20for%3F%22%2C%20%22axisLabelY%22%3A%20%22%20-%20count%22%2C%20%22horizontal%22%3A%20false%2C%20%22version%22%3A%201%2C%20%22metricColumnX%22%3A%20null%2C%20%22bucketColumn%22%3A%20%22d2%22%2C%20%22metricColumnY%22%3A%20null%2C%20%22showValueLabels%22%3A%20true%2C%20%22legendPosition%22%3A%20%22top%22%2C%20%22colors%22%3A%20%7B%22No%22%3A%20%22%23bf360c%22%2C%20%22Yes%22%3A%20%22%23388e3c%22%2C%20%22Not%20Sure%22%3A%20%22%23795548%22%7D%2C%20%22subBucketMethod%22%3A%20%22stack_percentage%22%2C%20%22axisLabelX%22%3A%20%22constituency-2%22%2C%20%22truncateSize%22%3A%20null%2C%20%22axisLabelYFromUser%22%3A%20false%7D'
# Flow.getData(instanceURL + testendpoint, Flow.getToken())

In [11]:
getLegend = lambda x : [y['key'] for y in x]
getLabel = lambda x : [y['label'] for y in x]
getData = lambda x : [y['value'] for y in x]
ordered_province = pd.DataFrame([
    {'ordered':'Central Makira'},
    {'ordered':'East Guadalcanal'},
    {'ordered':'Central Guadalcanal'},
    {'ordered':'East Honiara'},
    {'ordered':'Central Honiara'},
    {'ordered':'West Honiara'},
    {'ordered':'Aoke Langalanga'},
    {'ordered':'Small Malaita'},
    {'ordered':'Lau Mbaelelea'},
    {'ordered':'Russell Savo'},
    {'ordered':'Gao Bugotu'},
    {'ordered':'Maringe Kokota'},
    {'ordered':'Gizo Kolombangara'},
    {'ordered':'North Vella Lavella'},
    {'ordered':'North West Choiseul'}
])
ordered_province.reset_index(level=0, inplace=True)

In [12]:
def checkPercentage(allValues):
    index = 0
    diff = 0
    total = sum(allValues)
    if total < 100:
        index = allValues.index(min(allValues))
        diff = 100 - total
    elif sum(allValues) > 100:
        index = allValues.index(max(allValues))
        diff = 100 - total
    else:
        pass
    return [index,diff]

In [13]:
yesno = ['Yes','No']
def checkColorOrder(dataSeries, dataCommon, graphType, legends):
    opt = ['Yes', 'No']
    if set(['Not Sure']).issubset(legends):
        opt.insert(2, 'Not Sure')
    legend_df = pd.Series(legends).to_frame('legend').reset_index()
    opt_df = pd.Series(opt).to_frame('order').reset_index()
    opt_df = opt_df.merge(legend_df, left_on='order', right_on='legend', suffixes=['_a', '_b'], how='outer')
    ordered_legends = list(opt_df.sort_values(by='index_a')['index_b'])
    ordered_series = []
    ordered_keys = []
    for i in ordered_legends:
        if graphType == 'bar':
            ordered_series.append(dataSeries[i])
        else:
            ordered_series.append(dataSeries[0]['data'][i])
            ordered_keys.append(dataCommon['data'][i])
    transformed = {
        "series": ordered_series,
        "keys": ordered_keys
    }
    return transformed

In [14]:
def createBarChart(b,ids):
    ordered = ordered_province
    data = b
    output = r.get(server + templatedir + 'results-barchart.json').json()
    unordered = pd.DataFrame(data['common']['data'])
    unordered.reset_index(level=0, inplace=True)
    ordered_index = unordered.merge(ordered,left_on='key',right_on='ordered')
    ordered_index = ordered_index.sort_values(by=['index_y'], ascending=True)
    available_province = list(ordered_index['ordered'])
    ordered_index = list(ordered_index['index_x'])
    legends = getLegend(data['series'])
    if set(yesno).issubset(legends):
        transformed = checkColorOrder(data['series'], None, 'bar', legends)
        data['series'] = transformed['series']
    output['legend']['data'] = getLegend(data['series'])
    output['xAxis']['data'] = available_province
    for i,l in enumerate(output['legend']['data']):
        if i == 0:
            df = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})
        else:
            df[i] = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})[i]
    df['sum'] = df.sum(axis = 1, skipna = True)
    df['order'] = pd.Series(ordered_index)
    df = df.sort_values(by=['order'], ascending=True).reset_index()
    for i,l in enumerate(output['legend']['data']):
        df[i] = round(df[i] / df['sum'] * 100).astype(int)
    df = df.drop(columns=['sum'])
    series = list(range(len(output["legend"]["data"])))
    for i, rows in df.iterrows():
        new_list = []
        for a in series:
            new_list.append(rows[a])
        adjust = checkPercentage(new_list)
        df[adjust[0]][i] = df[adjust[0]][i] + adjust[1]
    output['series'] = []
    for i,l in enumerate(output["legend"]["data"]):
        dataList = [None if di is 0 else di for di in list(df[i])]
        output["series"].append({"name": l,
                                 "type": "bar",
                                 "stack": "total",
                                 "label": {
                                     "normal": {
                                         "fontSize": 20,
                                         "fontWeight": "bold",
                                         "show": True,
                                         "position": "inside",
                                         "formatter": "{c}%"
                                     }
                                },"data": dataList})
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + jsondir + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [15]:
def createPieChart(b,ids):
    data = b
    output = r.get(server + templatedir + 'results-piechart.json').json()
    legends = getLegend(data['common']['data'])
    if set(yesno).issubset(legends):
        transformed = checkColorOrder(data['series'], data['common'],'pie', legends)
        data['series'][0]['data'] = transformed['series']
        data['common']['data'] = transformed['keys']
    output['title']['text'] = str(sum(getData(data['series'][0]['data'])))
    output['series'][0]['data'] = []
    for i,d in enumerate(data['series'][0]['data']):
        output['series'][0]['data'].append({'value':d['value'],'name':data['common']['data'][i]['key']})
    getName = lambda x : [y['name'] for y in x]
    output['legend']['data'] = getName(output['series'][0]['data'])
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + jsondir + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [16]:
def generateChart(endpoint,charttype,ids,lumenid):
    cachename = lumenid + '.json'
    jsoncache = workdir + cachedir + cachename
    if os.path.exists(jsoncache):
        data = r.get(server + cachedir + cachename).json()
    else:
        data = Flow.getData(instanceURL + endpoint, Flow.getToken())
        f = open(jsoncache,"w")
        f.write(json.dumps(data))
        f.close()
    if charttype == 'pie':
        result = createPieChart(data,ids)
    else:
        result = createBarChart(data,ids)
    return result

In [17]:
# Debug
# ds = ds.iloc[:5]

In [19]:
ds['json'] =  ds.apply(lambda x: generateChart(x['endpoint'],x['type'],x['id'],x['lumenid']), axis=1)

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
ds

,id,name,endpoint,type,lumenid,json
0,2.3,Citizens asked to register in another constitu...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,pie,5cd41572-242c-4a06-8f4f-c283376d8689,2-3
1,4.4,Winning candidate will find out who citizens v...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c997c2d-d4fa-4492-9b6c-348d590c3472,4-4
2,2.25.B,Would you vote for a woman candidate? (All con...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c998616-f56f-40b5-b806-2eb3a3e12801,2-25-B
3,4.21,Overall assessment of voter conduct at polling...,/aggregation/5c9c1e4d-acf7-4b88-92b0-4a06c9be5...,bar,5cd8f90c-68e5-42e8-a88a-c34621108176,4-21
4,2.12,Have you seen ANY SIEC media/awareness (By con...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c99bbe4-d49a-47ef-a143-4e40994adc40,2-12
5,4.15,Polling station accommodates needs of those wi...,/aggregation/5c9c1e4d-acf7-4b88-92b0-4a06c9be5...,bar,5c9c1ea6-11c9-4644-a29d-6257debcf075,4-15
6,2.15.B,Have you received SIEC awareness via radio? (A...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c9c11ef-166d-4484-bb3a-c924bd9d45aa,2-15-B
7,2.14.B,Have you received SIEC awareness via text mess...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c9c106c-2646-4b69-9a68-d57bea0a4586,2-14-B
8,2.16.B,Have you received SIEC awareness via newspaper...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,bar,5c9c1129-9ae9-4563-b008-e65aed26152f,2-16-B
9,4.3,Winning candidate will find out who citizens v...,/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2...,pie,5c9c803c-099b-4013-8d6c-1149551dffd0,4-3


In [ ]:
# Debug
# skipchart = ['2.19','4.7.A','4.7.B','2.14','2.18.A','2.18.B']
# print(list(ds[ds['id'].isin(skipchart)]['name']))

In [23]:
def takeScreenShot(x,y):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('http://localhost:4000/#'+x)
    sleep(3)
    imagename = x + '-' + y.lower().replace('?',' ').replace('(','').replace(')','').replace(' ','-').replace('/','-')
    driver.save_screenshot('screenshots/'+ imagename +'.png')
    driver.close()
    return imagename + '.png'

In [ ]:
# Debug
# takeScreenShot('5-9','Candidates Counting Agents Influence Counting Process')

In [24]:
for data in ds.to_dict('records'):
    takeScreenShot(data['json'],data['name'])

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chr

/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  
/Users/dedenbangkit/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chr

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=78.0.3904.97)
